In [1]:
import pandas as pd

In [8]:
data = pd.read_csv('data/XYZCorp_LendingData.csv')

/var/folders/gz/zn68fqp56794cn97b73c30pw0000gn/T/ipykernel_47377/1445386200.py:1: DtypeWarning: Columns (17,45,53) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/XYZCorp_LendingData.csv')


In [11]:
data['term'].value_counts()

 36 months    600221
 60 months    255748
Name: term, dtype: int64